In [12]:
%%capture
!pip install transformers[sentencepiece]

In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
from transformers import pipeline

In [ ]:
# loading aspect-based sentiment analysis model
absa_tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")
absa_model = AutoModelForSequenceClassification \
  .from_pretrained("yangheng/deberta-v3-base-absa-v1.1")

#loading model from huggingface
sentiment_model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_model = pipeline("sentiment-analysis", model=sentiment_model_path,
                          tokenizer=sentiment_model_path)


In [34]:
def absa_sentiment(sentence : str, aspect1 : str, aspect2 : str):
  print(f"Sentence : {sentence}")
  print()
  labels = ["negative", "neutral", "positive"]

  inputs = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect1} [SEP]",return_tensors="pt")
  outputs = absa_model(**inputs)
  probs = F.softmax(outputs.logits, dim=1).detach().numpy()[0]
  print(f"Sentiment of aspect '{aspect1}' is : ")
  max_prob_index = probs.argmax()
  max_label = labels[max_prob_index]
  max_prob = probs[max_prob_index]
  print(f"Label {max_label}: {max_prob:.4f}")
  print()

  inputs = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect2} [SEP]", return_tensors="pt")
  outputs = absa_model(**inputs)
  probs = F.softmax(outputs.logits, dim=1)
  probs = probs.detach().numpy()[0]
  print(f"Sentiment of aspect '{aspect2}' is : ")
  max_prob_index = probs.argmax()
  max_label = labels[max_prob_index]
  max_prob = probs[max_prob_index]
  print(f"Label {max_label}: {max_prob:.4f}")
  print()

  sentiment = sentiment_model([sentence])[0]
  print(f"Overall sentiment: {sentiment['label']} with score {sentiment['score']}")

In [35]:
sentence = "The company's pricing strategy is commendable, offering excellent value for money. Their competitive rates and transparent fee structure make their products accessible to a wide range of customers without compromising on quality. However, the execution of their services leaves much to be desired. Delivery times are consistently longer than promised, customer support is frustratingly unresponsive, and the product often arrives in a condition that suggests careless handling during shipping. While the price point is attractive, the subpar execution significantly diminishes the overall customer experience, making one question whether the savings are worth the headache."

aspect1 = "pricing"
aspect2 = "execution"
absa_sentiment(sentence, aspect1, aspect2)

Sentence : The company's pricing strategy is commendable, offering excellent value for money. Their competitive rates and transparent fee structure make their products accessible to a wide range of customers without compromising on quality. However, the execution of their services leaves much to be desired. Delivery times are consistently longer than promised, customer support is frustratingly unresponsive, and the product often arrives in a condition that suggests careless handling during shipping. While the price point is attractive, the subpar execution significantly diminishes the overall customer experience, making one question whether the savings are worth the headache.

Sentiment of aspect 'pricing' is : 
Label positive: 0.9530

Sentiment of aspect 'execution' is : 
Label negative: 0.9809

Overall sentiment: positive with score 0.6510158777236938
